In [1]:
import pandas as pd

# gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Set notebook options ##

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.precision', 3)

## Scrape data if not already downloaded ##

In [4]:
######

## Read in and combine data ##

In [5]:
caitlyn  = pd.read_csv('caitlyn.csv' , low_memory=False)
kendall  = pd.read_csv('kendall.csv' , low_memory=False)
kylie    = pd.read_csv('kylie.csv'   , low_memory=False)
kim      = pd.read_csv('kim.csv'     , low_memory=False)
kourtney = pd.read_csv('kourtney.csv', low_memory=False)
khloe    = pd.read_csv('khloe.csv'   , low_memory=False)
kris     = pd.read_csv('kris.csv'    , low_memory=False)
rob      = pd.read_csv('rob.csv'     , low_memory=False)

FileNotFoundError: [Errno 2] File b'caitlyn.csv' does not exist: b'caitlyn.csv'

In [ ]:
kardashians = pd.concat([caitlyn, kendall, kylie, kim, kourtney, khloe, kris, rob])